Project to Obtain and download/manipulate the results of Sentinel 1 data

Creating an RBG of the study area

In [1]:
# import requirements
import asf_search as asf

Obtaining and printing the results of Sentinel 1 data

In [2]:
results = asf.search(platform=asf.PLATFORM.SENTINEL1, maxResults=5)

In [3]:
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'maxResults': 5
}
#setting up search parameter options in a dictionary
results = asf.search(**opts)
#print(results)

Geographic Searches!

In [4]:
#using a Well Knon Text (WKT) variable to define a polygon for the search to use
#It is here I will attempt to use a shapefile or similar that someone can input to define the area, maybe even a search fora city name?
geo_results = asf.geo_search(
        intersectsWith='POLYGON((-152.81 58.49,-154.90 57.49,-155.08 56.30,-153.82 56.34,-151.99 57.30,-151.43 58.19,-152.81 58.49))',
        platform=asf.PLATFORM.ALOS,
        start='2010-01-01',
        end='2010-02-01')

In [5]:
print(f'{len(geo_results)} results found')

137 results found


In [6]:
#reduce the amount of results by using a centroid of a shape/scene
centroid = results[0].centroid().wkt

centroid_results = asf.geo_search(intersectsWith=centroid, **opts)

print(f'{len(centroid_results)} results found')

5 results found


Performing a Product Search<br>
See [Vertex](https://search.asf.alaska.edu/#/?zoom=7.370&center=-134.548,55.753&resultsLoaded=true&searchType=List%20Search&searchList=S1A_IW_GRDH_1SDV_20190809T001336_20190809T001401_028485_033839_78A1-GRD_HD,S1A_IW_GRDH_1SDV_20150322T000454_20150322T000524_005137_006794_56E3-GRD_HD,S1A_IW_GRDH_1SDV_20160121T001256_20160121T001321_009585_00DF26_5B84-GRD_HD,S1A_IW_GRDH_1SDV_20151117T000448_20151117T000513_008637_00C455_3DC2-GRD_HD&listSearchType=Product)

In [7]:
product_list = [
    'S1A_IW_GRDH_1SDV_20190809T001336_20190809T001401_028485_033839_78A1-GRD_HD',
    'S1A_IW_GRDH_1SDV_20150322T000454_20150322T000524_005137_006794_56E3-GRD_HD',
    'S1A_IW_GRDH_1SDV_20160121T001256_20160121T001321_009585_00DF26_5B84-GRD_HD',
    'S1A_IW_GRDH_1SDV_20151117T000448_20151117T000513_008637_00C455_3DC2-GRD_HD'
]
#Prodcuts were found using ASF's Vertex online app 

In [8]:
results = asf.product_search(product_list)

print(f'{len(results)} results found')

4 results found


Baseline Searches - Creating image stacks<br>
Not needed for me


### Downloading Data
#### Before You Start
The steps outlined in this demonstration assume `asf_search` is available in your environment. For guidance on installing `asf_search`, [begin here](./1-Basic_Overview.ipynb#Before-You-Start).

Additionally, this section expects you to have an [Earthdata Login](https://urs.earthdata.nasa.gov/) account with the appropriate applications authorized, EULAs signed, and profile fields set. The easiest way to check that your EDL account is in order is to simply go to [Vertex](https://search.asf.alaska.edu) and download a product.

In [9]:
from pathlib import Path
dirs = ['downloads']

In [4]:
import asf_search as asf
import getpass

username = input('Username:')
password = getpass.getpass('Password:')

try:
    user_pass_session = asf.ASFSession().auth_with_creds(username, password)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

Success!


`auth_with_token()`<br>
This authentication method accepts an EDL Token which is then included as part of an `Authorization: Bearer` header on any downloads using this session. To generate an EDL Token, [sign in to EDL](https://urs.earthdata.nasa.gov/home), select the "Generate Token" tab, and then click the green "Generate Token" button. The token can then be copied and used below.
  
__Note:__ While it is extremely convenient, not all datapool hosts are compatible with this authentication method yet.

In [ ]:
import getpass
token = getpass.getpass('EDL Token:')

token_session = asf.ASFSession().auth_with_token(token)

 `auth_with_cookiejar()`<br>
This method accepts an `http.cookiejar` compatible object, such as a previously authenticated session stored for later re-use.

For this demonstration, we will make use of the cookiejar from one of the previously authenticated sessions above:

In [ ]:
cookiejar = user_pass_session.cookies #take username and password session cookies

In [ ]:
cookiejar_session = asf.ASFSession().auth_with_cookiejar(cookiejar)

***
## Downloading
[View this search in Vertex](https://search.asf.alaska.edu/#/?dataset=UAVSAR&productTypes=METADATA&resultsLoaded=true&zoom=8.090&center=-90.488,28.359&polygon=POLYGON((-91.97%2028.78,-88.85%2028.78,-88.85%2030.31,-91.97%2030.31,-91.97%2028.78)))
  
With authentication handled, we can now begin downloading products. First, we will need some search results to work with:

In [13]:
import os.path
from os import mkdir

downPath = './downloads'
isDir = os.path.isdir(downPath)

if isDir == False:
    mkdir('downloads')

In [6]:
#get metadata search results as a quick example
results = asf.geo_search(
    intersectsWith='POLYGON((-91.97 28.78,-88.85 28.78,-88.85 30.31,-91.97 30.31,-91.97 28.78))',
    platform=asf.PLATFORM.UAVSAR,
    processingLevel=asf.PRODUCT_TYPE.METADATA,
    maxResults=250)

print(f'{len(results)} results found')

250 results found


In [7]:
from os import listdir

results[0].download(path='./downloads', session=user_pass_session)

listdir('./downloads')

['atchaf_19809_21056_009_210913_L090_CX_01.ann']

In [8]:
results[0:10].download(path='./downloads', session=user_pass_session)
listdir('./downloads')

c:\Users\Kyle\OneDrive\School\Concordia\8_W22\GEOG_464_GIS_Programming\464TermProject\.venv_464Project\lib\site-packages\asf_search\download\download.py:61: UserWarning: File already exists, skipping download: ./downloads\atchaf_19809_21056_009_210913_L090_CX_01.ann
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')


['atchaf_06309_21056_004_210913_L090_CX_01.ann',
 'atchaf_06309_21056_006_210913_L090_CX_01.ann',
 'atchaf_06309_21056_008_210913_L090_CX_01.ann',
 'atchaf_19809_21056_003_210913_L090_CX_01.ann',
 'atchaf_19809_21056_005_210913_L090_CX_01.ann',
 'atchaf_19809_21056_007_210913_L090_CX_01.ann',
 'atchaf_19809_21056_009_210913_L090_CX_01.ann',
 'pfourc_13320_21055_023_210912_L090_CX_01.ann',
 'wterre_16300_21055_021_210912_L090_CX_01.ann',
 'wterre_34202_21055_022_210912_L090_CX_01.ann']